In [3]:
import eelbrain as eel
import numpy as np
import cochlea, tqdm, pathlib, time
from pathnames import *

# load erg and save

In [ ]:
datadict = eel.load.unpickle(preprocessed_path / f'part_001_in_speech_Cz_reref_ergs.pkl')
ergs = [e.sub(time=(1, 245)) for e in datadict['ergs']]
ergs = eel.combine([e-e.mean() for e in ergs])

for i in tqdm(range(8)):
    wav = ergs[i].copy()
    wav /= wav.abs().max()
    eel.save.wav(wav, predictor_folder / f'part_001_in_speech_erg_short_wav_{i}.wav')
    eel.save.pickle(wav, predictor_folder / f'part_001_in_speech_erg_short_wav_{i}.pkl')
    wav = eel.resample(eel.NDVar(wav.x, eel.UTS(1, wav.time.tstep, len(wav))), 16384)

# generate zilany (only hsr fibres)

In [ ]:
fs_cochlea = 100000
cfs = 2 ** np.arange(np.log2(125), np.log2(16e3 + 1), 1./6)

print('running')
tts = []
ANps = []
ANns = []
for i in tqdm.tqdm(range(8)):
    wav = eel.load.unpickle(predictor_folder / f'part_001_in_speech_erg_wav_{i}.pkl')
    wav1 = eel.resample(wav, fs_cochlea)
    wav1.x = 0.0796214341107 * wav1.x / wav1.rms() # 72 dB SPL -> Pascal
    for ip, ee in enumerate([wav1.x, -wav1.x]):
        rr = []
        ts = time.time()
        for cf in tqdm.tqdm(cfs):
            rr1 = cochlea.run_zilany2014_rate(ee,
                                        fs_cochlea,
                                        anf_types=['hsr'],
                                        cf=cf,
                                        powerlaw='approximate',
                                        species='human')
            rr.append(rr1)
        rr = np.mean(np.squeeze(np.asarray(rr)), axis=0)
        tts.append(time.time()-ts)
        rr = eel.NDVar(rr, eel.UTS(wav.time.tstart, 1/fs_cochlea, len(rr)))
        rr = eel.resample(rr, 1/wav.time.tstep)
        print(rr, rr.time.tmax, f'elapsed {tts[-1]}\n')
        eel.save.pickle(rr, predictor_folder / f'part_001_in_speech_erg_short_wav_{i}_zilany_hsr_approx_pos{ip}.pkl')
        if ip: ANns.append(rr) 
        else: ANps.append(rr)

eel.save.pickle([ANps, ANns], predictor_folder / f'part_001_in_speech_erg_short_wav_zilany_hsr_posneg_all.pkl')